In [1]:
import cv2 as cv2
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import trackpy as tp
from ipywidgets import HBox, Textarea, interact
import time
from multiprocessing import Pool,process

def l2n(x):
    return np.array(x)

In [2]:
def parseCircles(circles):
    x = []
    y = []
    r = []
    for i in range(len(circles)):
        x.append(circles[i,0,0])
        y.append(circles[i,0,1])
        r.append(circles[i,0,2])
        
    x,y,r = l2n(x), l2n(y),l2n(r)
    return x,y,r

In [3]:
def preProcessFrame(frame):
    #blur and grayout a frame
    
    # Convert to grayscale.
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
      
    # Convert to HSV and extract the saturation channel
    #sat = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)[:,:,0]

    # median filter
    #median = cv2.medianBlur(gray, 3)
    #gray_blurred = median
    # Blur using 3 * 3 kernel.
    blurSize = 10#5
    gray_blurred = cv2.blur(gray, (blurSize, blurSize))
    
    return gray_blurred

def detectCircles(frame,minRadius,maxRadius,param1=30,param2=25):
    
    minDist = int(2.5*maxRadius)
    dp = 1
    # Apply Hough transform on the blurred image.
    circles = cv2.HoughCircles(frame, 
                cv2.HOUGH_GRADIENT,dp, minDist, param1 = param1,
                param2 = param2, minRadius = minRadius, maxRadius = maxRadius)

    return circles
    
def captureFrame(video,frame_id):
    video.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
    ret, frame = video.read()
    if ret:
        return frame
    else:
        return ret

def drawCircles(circles,ax ):
    if circles is not None:  
    # Convert the circle parameters a, b and r to integers.
    #detected_circles = np.uint16(np.around(detected_circles))
    #fig, ax = plt.subplots() 
        for pt in circles[0, :]:
            a, b, r = pt[0], pt[1], pt[2]
            circle = plt.Circle((a, b), r, color='r',fill=False)
            ax.add_patch(circle)
            #ax.imshow(frame, cmap = 'gray')

def drawCirclesDf(df,ax ):
    
    
    for index, row in df.iterrows():
        x,y,r = row.x,row.y,row.r
        circle = plt.Circle((x, y), r, color='r',fill=False)
        ax.add_patch(circle)

def circlesToDataFrame(circles,frameNum):    
    x,y,r = parseCircles(circles)
    data = {'frame':[int(frameNum)]*len(circles),'x':x,'y':y,'r':r}
    dft = pd.DataFrame(data = data)
    
    return dft


def cropImage(img,x0,y0,width,height):
    return img[y0:y0+height,x0:x0+width,:]
'''def cropImage(img,x0,y0,width,height):
    d = len(img.shape)
    if d==2: #grayscale
        return img[y0:y0+height,x0:x0+width]
    elif d==3: #rgb
        return img[y0:y0+height,x0:x0+width,:]
'''

'def cropImage(img,x0,y0,width,height):\n    d = len(img.shape)\n    if d==2: #grayscale\n        return img[y0:y0+height,x0:x0+width]\n    elif d==3: #rgb\n        return img[y0:y0+height,x0:x0+width,:]\n'

### Experimental settings

In [4]:
arenaSizePixels = 1090 # [pixels]
arenaSizeCm = 50 # [cm]

### Load experiments summary table

In [5]:
experiments = pd.read_csv('C:\\Users\\User\\Documents\\Eden\\locate_git_code\\two_circles_exp\\two_circles_exp4.csv',index_col=0)#,ignore_index=True)

## Locating params 


In [6]:
x0,y0 = 300,0
width0,height0 = 1150,1080

#frameNum = 1500
threshMin = 10
threshMax = 300

param1 = 31#100#25#35#30#25#25
param2 = 34#35

#frameSkip = 25

#### filter a sub-set of experiments

In [7]:
#expToTrack = experiments[(experiments["thirdpassiveDiameter[cm]"]==28)]
#expToTrack = experiments.iloc[[30,31,32,33,34]]
expToTrack = experiments.iloc[[28]]
#expToTrack = experiments[(experiments["date"]==20231212)]

### Batch locate - will take a while

In [8]:
locatingFileNameSuffix = '_locatedFull20231218_28cm_Eden.csv'
len(expToTrack)

1

In [ ]:
if __name__ == "__main__":
    with Pool() as pool:
        globalStartTime = time.time()

        locatedFileNames = []


        for index,row in (expToTrack.iterrows()):
            fileName  =row.path
            print('Locating a circle in ' + fileName);
            video = cv2.VideoCapture(fileName)

            #get videos properties
            width  = int(video.get(cv2.CAP_PROP_FRAME_WIDTH ))
            height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT ))
            fps = video.get(cv2.CAP_PROP_FPS)      # OpenCV v2.x used "CV_CAP_PROP_FPS"
            frameCount = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
            pixelSize = arenaSizeCm/arenaSizePixels

            #defined according the column from the experiments table
            radiusMin = 148#int(row.minCircleRadius)
            radiusMax = 160#int(row.maxCircleRadius)


            data = {'frame':[],'x':[],'y':[],'r':[]}
            columns = ['frame','x','y','r']
            df = pd.DataFrame(data=data,columns=columns)

            boxSize = int(2*(2*radiusMax))
            wc,hc = boxSize,boxSize
            xc,yc = int(958-boxSize/2),int(548-boxSize/2)

            #xc,yc = 1184,634 frameMin=6710 #
            frameMin = 0 #0#frameCount-100
            frameMax = 10#frameCount#frameCount#frameMin+1000#frameCount#frameMin+500#300#frameCount#minFrame+3*60*24
            #frameRange = range(frameMin,frameMax,frameSkip)


            startTime = time.time()
            video.set(cv2.CAP_PROP_POS_FRAMES, frameMin)
            ret = True

            while ret:

                #video.grab()#captureFrame(video,frameNum)
                test=pool.map(video.read(),range(frameMin,frameMax))
                ret, img = video.read()
                if(ret): #if succesfffuly read an image
                    frameNum = video.get(cv2.CAP_PROP_POS_FRAMES)
                    #imgRed = img[:,:,0]

                    #For large images, locate a cropped window

                    #imgCropped = cropImage(img,xc,yc,wc,hc)
                    imgCropped =cropImage(img,300,0,1150,int(height))
                    #Edge detection:
                    #imgE = cv2.Canny(imgCropped,threshMin,threshMax)
                    GrayImage = cv2.cvtColor(imgCropped, cv2.COLOR_BGR2GRAY)
                    #Find circles:
                    #circles = detectCircles(imgE,radiusMin, radiusMax,param1,param2)
                    circles = detectCircles(GrayImage,radiusMin, radiusMax,param1,param2)
                    #If circle found, add to pandas Dataframe:
                    if circles is not None:        
                        dft = circlesToDataFrame(circles,frameNum)
                        #dft['x'] = dft['x']
                        #dft['y'] = dft['y']

                        df = pd.concat([df,dft],ignore_index=True)
              #Save to CSV
            endTime = time.time()
            duration = endTime-startTime
            print('Locating ' + str(len(df)) + ' frames took ' + str(duration) + ' seconds.')

            #Save the pandas locating dataframe a CSV file at the video's location
            fileNameLoc = fileName+locatingFileNameSuffix
            df.to_csv(fileNameLoc)        
            locatedFileNames.append(fileNameLoc)
        ####################################linking#################################

            tp.quiet()
            search_range= 80
            memory= 1000
            minTrajLength =int(frameCount/4) #This is not the best condition (Matan)
            t = tp.link(df, search_range=search_range, memory=memory)
            t1=tp.filter_stubs(t,threshold=minTrajLength)
            path=fileName+locatingFileNameSuffix+'_linked.csv'
            t1.to_csv(path)


        globalEndTime = time.time()
        totalDuration = globalEndTime-globalStartTime
        print('Locating ' + str(len(expToTrack)) + ' experiments took ' + str(totalDuration) + ' seconds.')

Locating a circle in D:\Eden\new_exp_test_15_11_23\12_12_23\C0646.MP4


In [ ]:
### Save locating and linking filenames int expToTrack dataframe
### In power cell, change fetching of locating/tracking from the table of experiments

In [ ]:
print('Locating ' + str(len(expToTrack)) + ' experiments took ' + str(totalDuration) + ' seconds.')

In [ ]:
print('There were ' + str(int(len(locatedFileNames))) + ' files located')
print('the total run duration was: ' + str(np.round(totalDuration/60/60,1)) +' hours')